In [8]:
import pandas as pd
df=pd.read_csv('/Users/cyrilmahfouz/Desktop/SEF/Week2/Pandas_final/kc_house_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

<h3><font color='red'>Introduction</font></h3>

In [9]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


To start,we are looking at a dataframe representing house sales in King County, USA from Kaggle. This dataframe contains information about 21613 houses. We notice that the dominant data type is the int64.Furthermore we can detect that all of these columns revolves around one target value and that is the price column. t.<br>
<br>
Let's begin by identifying the interesting columns and how could they possibly relate to the price.The logical links that affect the prices are:<br>
1- The number of bedrooms<br>
2- The area of the house, the number of floors<br>
3- The location of the house<br>
4- The conditon of the house and the grade that is was given<br>
5- The number of views and the features available in this house<br>

<h3><font color='red'>Anazlyze</font></h3>

<h4>The bedrooms</h4>

Lets take a look at the percentage of the houses that have 'X' bedrooms

In [19]:
perc_bedrooms=df.bedrooms.value_counts(normalize=True).mul(100)
house_nb_bedrooms=df.bedrooms.value_counts(normalize=False)

print("the percentage of the houses that contains x bedrooms: \n",perc_bedrooms)
print("the houses that contains x bedrooms: \n",house_nb_bedrooms)

the percentage of the houses that contains x bedrooms: 
 bedrooms
3     45.454125
4     31.841947
2     12.770092
5      7.407579
6      1.258502
1      0.920742
7      0.175820
0      0.060149
8      0.060149
9      0.027761
10     0.013881
11     0.004627
33     0.004627
Name: proportion, dtype: float64
the houses that contains x bedrooms: 
 bedrooms
3     9824
4     6882
2     2760
5     1601
6      272
1      199
7       38
0       13
8       13
9        6
10       3
11       1
33       1
Name: count, dtype: int64


We can notice several interesting values.First that there is a house that has an odd amout of bedrooms.This house contains 33 bedrooms that why we are going to dig deepeer to check if the area of the house is logical and the number of floors.Because it could be an error in the data inserted and we doesn't want it to distrupt our calculations.<br>
But before we start lets get some information about the prices

In [30]:
#the average price of these houses:
price_info=df['price'].agg({'mean', 'max'}).round()
print("prices info:\n ",price_info)

prices info:
  mean     540182.0
max     7700000.0
Name: price, dtype: float64


In [25]:
#lets locate the row that has this anomaly
anomaly_bed=df.loc[df.bedrooms==33]
print(anomaly_bed)

               id             date     price  bedrooms  bathrooms  \
15870  2402100895  20140625T000000  640000.0        33       1.75   

       sqft_living  sqft_lot  floors  waterfront  view  ...  grade  \
15870         1620      6000     1.0           0     0  ...      7   

       sqft_above  sqft_basement  yr_built  yr_renovated  zipcode      lat  \
15870        1040            580      1947             0    98103  47.6878   

          long  sqft_living15  sqft_lot15  
15870 -122.331           1330        4700  

[1 rows x 21 columns]


Seeing that the house that contains 33 bedroom has a small area, 1 floor and a price very far from the maximum price which doesnt make sense we can conclude that this information could possibly be wrong,so we will be deleting it before proceeding to the bedrooms grouping.

In [55]:
df_=df.copy().drop(15870)

#lets proceed by grouping the houses in 2 groups according to the bedrooms:
df_['has_4_or_more_bedrooms'] = df_['bedrooms'].apply(lambda x: 1 if x >= 4 else 0)
df_[['price','has_4_or_more_bedrooms','sqft_living','sqft_lot']].groupby('has_4_or_more_bedrooms').agg({'price':['mean', 'min','max'],
                                                                                                     'sqft_lot':['mean', 'min' ,'max'],
                                                                                                     'sqft_living':['mean', 'min' ,'max']}).round()


price                      sqft_lot                \
                            mean       min        max     mean  min      max   
has_4_or_more_bedrooms                                                         
0                       449912.0   75000.0  3800000.0  13954.0  572  1164794   
1                       671193.0  100000.0  7700000.0  16782.0  520  1651359   

                       sqft_living              
                              mean  min    max  
has_4_or_more_bedrooms                          
0                           1669.0  290   6840  
1                           2676.0  800  13540

Out of this data, we can see that the houses that has more than 4 bedrooms have a strong relation with the price.The houses with 4 or more rooms have an average price hogher that the others houses;which we can conclude that the more expensive prices are for the houses that has a larger number of bedrooms.Also we remark that the average area of these houses is much bigger than the houses that has smaller amount of bedrooms.<br>
In conclusion there is a lot of values related to the bedrooms and they could possibly affect the price variation.

<h4>The area</h4>

Now, after checking how the bedrooms impact the price of the house we can logically assume that the area of the houses also affect this value. So lets add a column called area to viualize it clearly infront of us.<br>


In [100]:
df_['area']=df_['sqft_living']+df_['sqft_lot']
avrg_area=df_.area.mean()

print("avrg area: ",avrg_area.round())
area_bedrooms=df_[['price','sqft_lot','sqft_living','area']].sort_values(by='price',ascending=False)

area_bedrooms['bigger_avrg_area']=area_bedrooms['area'].apply(lambda x:1 if x >avrg_area else 0 )


area_bedrooms.groupby('bigger_avrg_area').agg({'price' :['mean','min','max'],
                                                'sqft_lot':['min','max'],
                                                'sqft_living':['min','max'],
                                                'area':['mean','min','max']})



avrg area:  17187.0


price                     sqft_lot           \
                           mean      min        max      min      max   
bigger_avrg_area                                                        
0                 501207.989882  78000.0  3650000.0      520    16345   
1                 754275.874700  75000.0  7700000.0    10862  1651359   

                 sqft_living                 area                  
                         min    max          mean    min      max  
bigger_avrg_area                                                   
0                        370   6840   8886.591391   1423    17183  
1                        290  13540  62791.378005  17200  1652659

Lets analyze the result:<br>
For the houses that has 